<a href="https://colab.research.google.com/github/Riky2014/Tesi/blob/main/1d_hemo_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install software-properties-common
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install fenics

In [2]:
from fenics import *
import numpy as np
import matplotlib.pyplot as plt

In [42]:
# Data
A0 = 4e-4
q0 = 0
k_r = 2.416
rho = 1.05
alpha = 1

T = 1
x_left = 0
x_right = 1.0

# Discretization parameter
dt = 1e-5
h = 1e-2
num_steps = T / dt
N = int((x_right - x_left) / h)

#  Create a mesh on the interval [0, 1].
mesh = IntervalMesh(N, x_left, x_right)
x = MeshCoordinates(mesh)

# Define the function space
P1 = FiniteElement('P', mesh.ufl_cell(), 1)
element = MixedElement([P1, P1])
V = FunctionSpace(mesh, element)

In [43]:
# Exact solution
L = 1
A_tilde = 4e-4
a_tilde = 4e-5
q_tilde = 0.0
A_exact = Expression('A_tilde + a_tilde * sin(2 * pi / L * x[0]) * cos(2 * pi / T * t)', A_tilde = A_tilde, a_tilde = a_tilde, L = L, T = T, degree = 2, t = 0)
q_exact = Expression('q_tilde - (a_tilde * L / T) * cos(2 * pi / L * x[0]) * sin(2 * pi / T * t)', q_tilde = q_tilde, a_tilde = a_tilde, L = L, T = T,degree = 2, t = 0)

In [44]:
def U(A, q):
  return as_vector([A, q])

def H(A, q):
  return as_tensor([[0, 1], [1190476.19048 * A ** 0.5 - (q / A) ** 2, 2 * q / A]])

def F(A, q):
  return as_vector([q, 793650.79365 * A ** 1.5 - 6.34921 + q ** 2 / A])

def B(A, q):
  return as_vector([0, k_r * q / A])

def S(A, q):
  return as_vector([0, k_r * q / A])

def dS_dU(A, q):
  return as_tensor([[0, 0], [- k_r * q / A ** 2, k_r / A]])

In [45]:
def c_alpha(A, q, alpha = 1):
  return (1190476.19048 * A ** 0.5 + (q / A) ** 2 * alpha * (alpha - 1)) ** 0.5

def l1(A, q, alpha = 1):
  return as_vector([c_alpha(A, q, alpha) - alpha * q / A, 1])

def l2(A, q, alpha = 1):
  return as_vector([- c_alpha(A, q, alpha) - alpha * q / A, 1])

def CC(A, q):
  return U(A, q) - dt * dot(H(A, q), U(A, q).dx(0)) - dt * B(A, q) + dt * f

In [46]:
def inlet_bc(A, q):
  q_inlet = (
      (project(l2(A, q)[0] * CC(A, q)[0], V.sub(0).collapse()).compute_vertex_values(mesh)
      + project(l2(A, q)[1] * CC(A, q)[1], V.sub(1).collapse()).compute_vertex_values(mesh)
      - project(l2(A, q)[0] * U(A, q)[0], V.sub(0).collapse()).compute_vertex_values(mesh))
      / project(l2(A, q)[1] , V.sub(1).collapse()).compute_vertex_values(mesh)
  )[0]

  return q_inlet

In [47]:
def outlet_matrix(A, q):
  return as_tensor([[1, -1], [c_alpha(A, q, alpha) + alpha * q / A, c_alpha(A, q, alpha) - alpha * q / A]])

def outlet_vector(A, q):
  return as_vector([l1(A, q)[0] * CC(A, q)[0] + l1(A, q)[1] * CC(A, q)[1], l2(A, q)[0] * (U(A, q) - dt * S(A, q))[0] + l2(A, q)[1] * (U(A, q) - dt * S(A, q))[1]])

def outlet_bc(A, q):
  A_out_fun, q_out_fun = project(dot(outlet_matrix(A, q), outlet_vector(A, q)) / (2 * c_alpha(A, q)), V).split()
  A_outlet = A_out_fun.compute_vertex_values(mesh)[-1]
  q_outlet = q_out_fun.compute_vertex_values(mesh)[-1]

  return A_outlet, q_outlet

In [48]:
# Define the source term
f = Expression(('0','5.98399 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * cos(6.28319*t) * cos(6.28319*x[0]) - 0.00025 * cos(6.28319*t) * cos(6.28319*x[0]) - 0.0001 * sin(6.28319*t) * cos(6.28319*x[0]) / (4.0e-5 * sin(6.28319*x[0]) * cos(6.28319*t) + 0.0004)'), degree = 2, t = 0)
df_dt = Expression(('0', '-1.87993 * sin(6.28319*t) * sin(6.28319*x[0]) * cos(6.28319*t) * cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) - 37.5985 * pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 0.5) * sin(6.28319*t) * cos(6.28319*x[0]) + 0.00157 * sin(6.28319*t) * cos(6.28319*x[0]) - 0.15752 * pow(sin(6.28319*t), 2) * sin(6.28319*x[0])* cos(6.28319*x[0]) / pow((0.1 * sin(6.28319*x[0]) * cos(6.28319*t) + 1), 2) - 0.00063 * cos(6.28319*t) * cos(6.28319*x[0]) / (4.0e-5 * sin(6.28319*x[0]) * cos(6.28319*t) + 0.0004)'), degree = 2, t = 0)

f_n = interpolate(f, V)
df_dt_n = interpolate(df_dt, V)

In [49]:
# Define initial guess
u_old = interpolate(Expression(('A_tilde + a_tilde * sin(2 * pi / L * x[0])', 'q0'), degree = 2, A_tilde = A_tilde, a_tilde = a_tilde, L = L, q0 = q0), V)
A_old, q_old = u_old.split()

In [51]:
# Boundary condition
inlet = 'near(x[0], 0)'
outlet = 'near(x[0], 1)'
A_inlet = A0
q_inlet = inlet_bc(A_old, q_old)
A_outlet, q_outlet = outlet_bc(A_old, q_old)
print(f'A inlet = {A_inlet}, q inlet = {q_inlet}')
print(f'A outlet = {A_outlet}, q outlet = {q_outlet}')

# Dirichlet bc
bc_A_inlet = DirichletBC(V.sub(0), A_inlet, inlet)
# I should impose compatibility condition
bc_q_inlet = DirichletBC(V.sub(1), q_inlet, inlet)

# Non reflecting bc
bc_A_outlet = DirichletBC(V.sub(0), A_outlet, outlet)
# I should impose compatibility condition
bc_q_outlet = DirichletBC(V.sub(1), q_outlet, outlet)

bc = [bc_A_inlet, bc_q_inlet, bc_A_outlet, bc_q_outlet]

A inlet = 0.0004, q inlet = 1.72325385905392e-08
A outlet = 0.0004000000558389759, q outlet = 8.62327922319775e-09


In [52]:
# Define trial functions and test functions
A, q = TrialFunctions(V)
v, z = TestFunctions(V)

# Define the variational problem
a = inner(A, v) * dx + inner(q, z) * dx

L = (
      A_old * v * dx
    + q_old * z * dx
    - dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old)))).dx(0)[0] * v * dx
    - dt * ((F(A_old, q_old) - dt / 2 * dot(H(A_old, q_old), S(A_old, q_old)))).dx(0)[1] * z * dx
    + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)) + (dot(H(A_old, q_old), F(A_old, q_old).dx(0))).dx(0))[0] * v * dx
    + dt ** 2 / 2 * (dot(dS_dU(A_old, q_old), F(A_old, q_old).dx(0)) + (dot(H(A_old, q_old), F(A_old, q_old).dx(0))).dx(0))[1] * z * dx
    - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[0] * v * dx
    - dt * (S(A_old, q_old) - dt / 2 * dot(dS_dU(A_old, q_old), S(A_old, q_old)))[1] * z * dx
    + dt * f[0] * v * dx
    + dt * f[1] * z * dx
    + dt ** 2 / 2 * (- (dot(H(A_old, q_old), f_n)).dx(0) - dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[0] * v * dx
    + dt ** 2 / 2 * (- (dot(H(A_old, q_old), f_n)).dx(0) - dot(dS_dU(A_old, q_old), f_n) + df_dt_n)[1] * z * dx
)

In [53]:
# Time stepping
u = Function(V)
t = 0
i = 0
for n in range(int(num_steps) + 1):

  i +=1

  # Update time step
  t += dt
  f.t = t
  df_dt.t = t
  A_exact.t = t
  q_exact.t = t

  # Solve the problem
  solve(a == L, u, bc)
  A, q = u.split(deepcopy = True)

  # Compute errors
  if (i % 100 == 0):
    print(f'Interation {i} / {int(num_steps) + 1}')
    A_e = interpolate(A_exact, V.sub(0).collapse())
    q_e = interpolate(q_exact, V.sub(1).collapse())
    error = np.array([errornorm(A, A_e, 'L2') / norm(A_e, 'L2'), errornorm(q, q_e, 'L2') / norm(q_e, 'L2')])
    print(f't = %.4f: error = {error}' % (t))
    print()

  # Update previous solution
  A_old.assign(A)
  q_old.assign(q)

  q_inlet = inlet_bc(A_old, q_old)
  A_outlet, q_outlet = outlet_bc(A_old, q_old)

Interation 100 / 100000
t = 0.0010: error = [3.18079643e-04 1.30060327e+01]

Interation 200 / 100000
t = 0.0020: error = [5.99293437e-04 1.35656519e+01]

Interation 300 / 100000
t = 0.0030: error = [8.50929356e-04 1.29537576e+01]

Interation 400 / 100000
t = 0.0040: error = [1.07874556e-03 1.22359361e+01]

Interation 500 / 100000
t = 0.0050: error = [1.28655754e-03 1.15652489e+01]

Interation 600 / 100000
t = 0.0060: error = [1.47706756e-03 1.09645315e+01]

Interation 700 / 100000
t = 0.0070: error = [1.65230314e-03 1.04297577e+01]

Interation 800 / 100000
t = 0.0080: error = [1.81386413e-03 9.95165309e+00]

Interation 900 / 100000
t = 0.0090: error = [1.96306768e-03 9.52104585e+00]

Interation 1000 / 100000
t = 0.0100: error = [2.10103473e-03 9.13037053e+00]

Interation 1100 / 100000
t = 0.0110: error = [2.22874492e-03 8.77333174e+00]

Interation 1200 / 100000
t = 0.0120: error = [2.34707777e-03 8.44462609e+00]

Interation 1300 / 100000
t = 0.0130: error = [2.45678886e-03 8.15243691e+

KeyboardInterrupt: 

In [ ]:
A.compute_vertex_values()